In [1]:
%matplotlib inline
%load_ext heat

In [8]:
#%%heat
import multiprocessing as mp
from IPython.display import Image
from copy import copy,deepcopy
from numpy import pi
import numpy as np
from qutip import *
from qutip.qip.operations import *
from qutip.qip.circuit import QubitCircuit, Gate
import time
from itertools import combinations

def user_gate1():
# S gate
    mat = np.array([[1.,0],[0., 1.j]])
    return Qobj(mat, dims=[[2], [2]])

def user_gate2():
# T gate
    mat = np.array([[1.,0],[0., 0.707+0.707j]])
    return Qobj(mat, dims=[[2], [2]])

def qu(gnum,n): # pick the unitary from the gate set #error
    
    if gnum<2*n+1:
        if gnum%2==0:
            g_t= Gate("T",int((gnum-1)//2)) 
        else:
            g_t= Gate("SNOT", int((gnum-1)//2))
    elif gnum<2*n+n+1:
        #print((gnum-3*n)%n, (gnum-3*n+1)%n )
        g_t= Gate("CNOT",int((gnum-2*n)%n),int((gnum-2*n+1)%n))
    return g_t

def quc(x,n): # generate the given quantum circuit
    qc= QubitCircuit(n)
    for i in range(len(x)):
        if x[i]!=0:
            qc.add_gate(qu(x[i], n))
    prop = qc.propagators()
    return gate_sequence_product(prop).full()
    

#may be faster to apply inverse i.e. loop over numbers gmax**r in base gmax
# def list_circuits(gmax,r):# list the circuits
#     def list_c(*arrays): 
#         grid = np.meshgrid(*arrays)        
#         coord_list = [entry.ravel() for entry in grid]
#         points = np.vstack(coord_list).T
#         return points
#     aa = np.arange(gmax)
#     return list_c(*r*[np.arange(gmax)])

# below is time inefficient but memory efficient
# not optimal 28.8 vs 15.7

def base_conv(cirnum,gmax,r): 
    xx= np.zeros(r)
    num=np.copy(cirnum)
    for i in range(r): 
        xx[i]= num%gmax
        num//=gmax
    return xx



def bi(ii,n):
    bix = np.zeros(n)
    for i in range(n): 
        bix[i]= ii%2
        ii//=2
    return(bix)

def list_povm(n):
    l_povms=[]
    for qbits in range(n+1): 
            for pv in combinations(np.arange(n),n-qbits):
                pov = np.zeros(n)
                for iden in  range(len(pv)):
                    pov[pv[iden]]= 1
                povm = gene_povm(pov,n)
                l_povms.append(np.diag(povm))
    return(l_povms)
                

# below is time inefficient but memory efficient
#removed since, it is less optimal 4.67s vs 3.54s


def gene_povm(gnp,n): 
    pv = np.zeros(2**n)
    b = gnp
    pv[0]= 1
    for j in range(len(b)): 
        if b[j]==1:
            pv[2**j:2**(j+1)]= pv[:2**j]
    return(np.diag(pv)) 

            
                
                

def trqrho(q,rho):
    trq= 0
    trq+= np.sum(np.diag(q)*np.diag(rho),)
    return(trq)

def tr(q): 
    return(np.sum(q))

def opt_povm(r,rho,gmax,n,lst_povm,cnum):
    H=2**n
    if cnum==0:
        counter = 0
        prevtr = 2**n
        thistr = 0
        for povme in range(len(lst_povm)):
            povm= lst_povm[povme]
            trqq = trqrho(rho,povm)
            thistr = tr(povm)
            if trqq>=eta:
                counter+=1
                if thistr<H:
                    H= np.copy(thistr)
            if (thistr!=prevtr) and counter==0:
                prevtr=np.copy(thistr)
                break
            else: 
                prevtr=np.copy(thistr)
    else:
        bc = base_conv(cnum,gmax,r)
        U = quc(bc,n)
        sigma = U@rho@U.conj().T # use of einsum may make it faster? need to make this step faster? sparce?
        counter=0
        prevtr = 2**n
        thistr = 0
        for povme in range(len(lst_povm)):
            povm= lst_povm[povme]
            trqq = trqrho(sigma,povm)
            thistr = tr(povm)
            if trqq>=eta:
                counter+=1
                if thistr<H:
                    H= np.copy(thistr)
            if (thistr!=prevtr) and counter==0:
                prevtr=np.copy(thistr)
                break
            else: 
                prevtr=np.copy(thistr)
    return(H)

    
def H_eep(r,eta,rho,gmax,n):
    ndg_list=[]        
    for cnum in range(0,gmax**r):# this lists out for unique gates
        bc=base_conv(cnum,gmax,r)
        if np.all(bc[bc !=0]-bc[:len(bc[bc !=0])]==0):
            ndg_list = np.append(ndg_list,cnum)
    tt = np.nditer(ndg_list)
    lst_povm = list_povm(n)
    nprocs = mp.cpu_count()
    pool=mp.Pool(processes=nprocs)
    H_array= pool.starmap(opt_povm,[(r,rho,gmax,n,lst_povm,circuitnum) for circuitnum in tt])
    pool.close()
    H_min=min(H_array)
    print("effective entropy for eta="+str(eta)+" and r="+str(r) +" is "+ str(np.log(H_min)/np.log(2)))
    return(np.log(H_min)/np.log(2))

def H_ee(r,eta,rho,gmax,n):
    ndg_list=[]
    tt=0
        
    for cnum in range(0,gmax**r):# this lists out for unique gates
        bc=base_conv(cnum,gmax,r)
        if np.all(bc[bc !=0]-bc[:len(bc[bc !=0])]==0):
            ndg_list = np.append(ndg_list,cnum)
            
    lst_povm = list_povm(n)
    circi=0
    povmi=0
    H=2**n
    for cnum in np.nditer(ndg_list):
        if cnum==0:
            counter = 0
            prevtr = 2**n
            thistr = 0
            for povme in range(len(lst_povm)):
                povm= lst_povm[povme]
                trqq = trqrho(rho,povm)
                thistr = tr(povm)
                if trqq>=eta:
                    counter+=1
                    if thistr<H:
                        circi = np.copy(cnum)
                        povmi= povme
                        H= np.copy(thistr)
                if (thistr!=prevtr) and counter==0:
                    prevtr=np.copy(thistr)
                    break
                else: 
                    prevtr=np.copy(thistr)
        else:
            bc = base_conv(cnum,gmax,r)
            U = quc(bc,n)
            sigma = U@rho@U.conj().T # use of einsum may make it faster? need to make this step faster? sparce?
            counter = 0
            prevtr = 2**n
            thistr = 0
            for povme in range(len(lst_povm)):
                povm= lst_povm[povme]
                trqq = trqrho(sigma,povm)
                thistr = tr(povm)
                if trqq>=eta:
                    counter+=1
                    if thistr<H:
                        circi = np.copy(cnum)
                        povmi = povme
                        H= np.copy(thistr)
                if (thistr!=prevtr) and counter==0:
                    
                    prevtr=np.copy(thistr)
                    break
                else: 
                    prevtr=np.copy(thistr)

    print("effective entropy for eta="+str(eta)+" and r="+str(r) +" is "+ str(np.log(H)/np.log(2)))
    print(povmi)
    print(circi)
    return(np.log(H)/np.log(2))

In [10]:
# All the inputs are in this box 

# system specific data
n = 3 # number of qubits
gmax= 1+2*n+n #number of unique gates in the universal gate set [2*n 1-qubit gates, n CNOT gates]
# Need to change qu() if this is changed

# process specific

eta = 0.8 # success rate? 
# define rho below this
x = np.random.rand(2**n,2**n)
rho= x@x.transpose()
rho = rho/np.trace(rho)
#H_ee(r,eta,rho,gmax,n)
np.savetxt("rh.txt",rho)

In [ ]:
r =5
eta = 0.8
rho1 = np.loadtxt("rh.txt")
%time H_eep(r,eta,rho1,gmax,n)

In [36]:
r =5
eta = 0.8
%time H_ee(r,eta,rho1,gmax,n)

effective entropy for eta=0.8 and r=5 is 2.0
5
2859.0
CPU times: user 12min 30s, sys: 1.01 s, total: 12min 31s
Wall time: 12min 31s


2.0

In [39]:
reps= 30
eta = 0.8 
r = 5
rho1 = np.loadtxt("rho1.txt")
base= 2#H_ee(r,eta,rho1,gmax,n)
histo = np.zeros(reps)
for haarunit in range(reps):
    qb1= np.random.randint(n)
    qb2 = np.random.choice([i for i in range(n) if i!= qb1])
    qc1 = QubitCircuit(n)
    qc1.user_gates={"RANDGT": rand_unitary_haar(4, [[2,2],[2,2]])}
    qc1.add_gate("RANDGT",targets=[qb1,qb2])
    props = qc1.propagators()
    V= gate_sequence_product(props).full()
    rho= V@rho1@V.conj().T
    histo[haarunit]= H_ee(r,eta,rho,gmax,n)-base
    print(histo[haarunit])
    np.savetxt('hist2.txt',histo)

np.savetxt('hist2.txt',histo)   

effective entropy for eta=0.8 and r=5 is 3.0
3
5.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
4
0.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
3
5.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
2
3.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
1
1.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
2
3.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
2
3.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
1
1.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
2
3.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
3
5.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
2
3.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
1
1.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
3
5.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
3
5.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
1
1.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
2
3.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
1
1.0
1.0
effective entropy for eta=0.8 and r=5 is 3.0
1
1.0
1.0
effective 

In [33]:
Q = rand_unitary_haar(4).full()
print(Q@Q.conj().T)

[[ 1.00000000e+00+0.00000000e+00j -5.55111512e-17+1.11022302e-16j
   5.55111512e-17+4.16333634e-17j  5.55111512e-17-5.46437895e-17j]
 [-5.55111512e-17-1.11022302e-16j  1.00000000e+00+0.00000000e+00j
   8.32667268e-17-5.55111512e-17j  1.43982049e-16-1.11022302e-16j]
 [ 5.55111512e-17-4.16333634e-17j  8.32667268e-17+5.55111512e-17j
   1.00000000e+00+0.00000000e+00j  1.94289029e-16+8.32667268e-17j]
 [ 5.55111512e-17+5.46437895e-17j  1.43982049e-16+1.11022302e-16j
   1.94289029e-16-8.32667268e-17j  1.00000000e+00+0.00000000e+00j]]
